In [ ]:
!apt install python3.7
!pip install -U torchtext==0.6.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.7 is already the newest version (3.7.13-1+bionic3).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en
!python -m spacy download de

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 32.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 68.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [ ]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize= tokenize_de,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True,
            batch_first = True)

TRG = Field(tokenize= tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True,
            batch_first = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts= ('.de', '.en'),
                                                    fields = (SRC, TRG))

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
)

In [ ]:

class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환        
    def forward(self, src):
        # src: [단어 개수, 배치 크기]: 각 단어의 인덱스(index) 정보
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

        # 문맥 벡터(context vector) 반환
        return outputs,hidden, cell


In [ ]:

class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()


        self.embedding = nn.Embedding(output_dim, embed_dim)


        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_ratio)

        self.hidden_size = hidden_dim
        self.attn = nn.Linear(self.hidden_size * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)
        self.attn_reshapenn= nn.Linear(embed_dim+hidden_dim,embed_dim)

    def attention(self,encoder_outputs,hidden):
      timestep = encoder_outputs.size(0)
      h = hidden[0].repeat(timestep, 1, 1).transpose(0, 1)
      encoder_outputs = encoder_outputs.transpose(0, 1)
      energy = F.relu(self.attn(torch.cat([h, encoder_outputs], 2)))
      energy = energy.transpose(1, 2)  # [B*H*T]
      v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [B*1*H]
      energy = torch.bmm(v, energy).squeeze(1) 
      return F.softmax(energy, dim=1).unsqueeze(1)



    def forward(self, input,enc_output, hidden, cell):
        input = input.unsqueeze(0)
        # input: [단어 개수 = 1, 배치 크기]
        
        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        dec_attn = self.attention(enc_output,hidden)
        
        context =dec_attn.bmm(enc_output.transpose(0, 1))
        context = torch.permute(context,(1,0,2))

        combined_input = torch.cat([context,embedded],dim=2)
        combined_input = self.attn_reshapenn(combined_input)
        output, (hidden, cell) = self.rnn(combined_input, (hidden, cell))

        prediction = self.fc_out(output.squeeze(0))

        return prediction, hidden, cell,dec_attn


In [ ]:

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        enc_output,hidden, cell = self.encoder(src)

        trg_len = trg.shape[0] 
        batch_size = trg.shape[1] 
        trg_vocab_size = self.decoder.output_dim 
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        input = trg[0, :]


        for t in range(1, trg_len):
            output, hidden, cell,attention = self.decoder(input,enc_output, hidden, cell)

            outputs[t] = output 
            top1 = output.argmax(1) 

            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 
        
        return outputs,attention


In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [ ]:
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (attn): Linear(in_features=1024, out_features=512, bias=True)
    (attn_reshapenn): Linear(in_features=768, out_features=256, bias=True)
  )
)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train() 
    epoch_loss = 0
    

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        trg =torch.permute(trg,(1,0))
        src =torch.permute(src,(1,0))
        output,attention = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)

        trg = trg[1:].reshape(-1)

        loss = criterion(output, trg)
        loss.backward() 

        optimizer.step()
        

        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval() 
    epoch_loss = 0
    
    with torch.no_grad():

        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            trg =torch.permute(trg,(1,0))
            src =torch.permute(src,(1,0))


            output,attention = model(src, trg, 0)

            output_dim = output.shape[-1]
            

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].reshape(-1)


            loss = criterion(output, trg)


            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() 

    train_loss = train(model, train_iterator, optimizer, criterion)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() 
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f}')

KeyboardInterrupt: ignored

In [ ]:
#model.load_state_dict(torch.load('tut4-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f}')

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"Entire tokens: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"Entire source index: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        enc_out,hidden, cell = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)

    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell,attention = model.decoder(trg_tensor,enc_out, hidden, cell)
            attentions[i] = attention

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) 

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:],attentions[:len(trg_tokens)-1]

In [ ]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    
    x_ticks = [''] + ['<sos>'] + [t.lower() for t in sentence] + ['<eos>']
    y_ticks = [''] + translation
     
    ax.set_xticklabels(x_ticks, rotation=45)
    ax.set_yticklabels(y_ticks)

    #ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    #ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [ ]:
example_idx = 6

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation,attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
display_attention(src, translation, attention)

In [ ]:
example_idx = 10

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

translation,attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')
display_attention(src, translation, attention)

In [ ]:
example_idx = 13

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

translation,attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')
display_attention(src, translation, attention)

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        

        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])

    
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

## B 

In [ ]:

class BEncoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embed_dim)


        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.GRU(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        

        self.dropout = nn.Dropout(dropout_ratio)
 
    def forward(self, src):

        embedded = self.dropout(self.embedding(src))


        outputs, hidden = self.rnn(embedded)


        return outputs,hidden, 


In [ ]:
class BDecoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()


        self.embedding = nn.Embedding(output_dim, embed_dim)


        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.GRU(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_ratio)

        self.hidden_size = hidden_dim
        self.attn = nn.Linear(self.hidden_size * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)
        self.attn_reshapenn= nn.Linear(embed_dim+hidden_dim,embed_dim)

    def attention(self,encoder_outputs,hidden):
      timestep = encoder_outputs.size(0)
      h = hidden[0].repeat(timestep, 1, 1).transpose(0, 1)
      encoder_outputs = encoder_outputs.transpose(0, 1)
      energy = F.relu(self.attn(torch.cat([h, encoder_outputs], 2)))
      energy = energy.transpose(1, 2)  # [B*H*T]
      v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [B*1*H]
      energy = torch.bmm(v, energy).squeeze(1) 
      return F.softmax(energy, dim=1).unsqueeze(1)



    def forward(self, input,enc_output, hidden):
        input = input.unsqueeze(0)

        
        embedded = self.dropout(self.embedding(input))


        dec_attn = self.attention(enc_output,hidden)
        
        context =dec_attn.bmm(enc_output.transpose(0, 1))
        context = torch.permute(context,(1,0,2))

        combined_input = torch.cat([context,embedded],dim=2)
        combined_input = self.attn_reshapenn(combined_input)
        output, hidden = self.rnn(combined_input, hidden)

        prediction = self.fc_out(output.squeeze(0))

        return prediction, hidden,dec_attn


In [ ]:
class BSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        enc_output,hidden = self.encoder(src)

        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        input = trg[0, :]


        for t in range(1, trg_len):
            output, hidden,attention = self.decoder(input,enc_output, hidden)


            outputs[t] = output
            top1 = output.argmax(1) 


            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1
        
        return outputs,attention


In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

enc = BEncoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = BDecoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

bmodel = BSeq2Seq(enc, dec, device).to(device)

In [ ]:
bmodel.apply(init_weights)
optimizer = optim.Adam(bmodel.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(bmodel, train_iterator, optimizer, criterion)
    valid_loss = evaluate(bmodel, valid_iterator, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(bmodel.state_dict(), 'Cseq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f}')

Epoch: 01 | Time: 0m 59s
	Train Loss: 5.342
	Validation Loss: 5.007
Epoch: 02 | Time: 0m 57s
	Train Loss: 5.097
	Validation Loss: 4.996
Epoch: 03 | Time: 0m 57s
	Train Loss: 5.047
	Validation Loss: 4.998
Epoch: 04 | Time: 0m 58s
	Train Loss: 4.994
	Validation Loss: 5.247
Epoch: 05 | Time: 0m 59s
	Train Loss: 4.966
	Validation Loss: 5.324
Epoch: 06 | Time: 0m 58s
	Train Loss: 4.874
	Validation Loss: 5.072
Epoch: 07 | Time: 0m 58s
	Train Loss: 4.674
	Validation Loss: 4.943
Epoch: 08 | Time: 0m 58s
	Train Loss: 4.492
	Validation Loss: 4.885
Epoch: 09 | Time: 0m 58s
	Train Loss: 4.325
	Validation Loss: 4.641
Epoch: 10 | Time: 0m 59s
	Train Loss: 4.202
	Validation Loss: 4.619


In [ ]:
#model.load_state_dict(torch.load('tut4-model.pt'))

test_loss = evaluate(bmodel, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f}')

| Test Loss: 4.587


In [ ]:
def translate_sentence_vb(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() 

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"Entire tokens: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"Entire source index: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        enc_out,hidden = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)

    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden,attention = model.decoder(trg_tensor,enc_out, hidden)
            attentions[i] = attention

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) 

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:],attentions[:len(trg_tokens)-1]

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu_vb(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence_vb(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score__ = calculate_bleu_vb(test_data, SRC, TRG, bmodel, device)

print(f'BLEU score = {bleu_score__*100:.2f}')

Entire tokens: ['<sos>', 'ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.', '<eos>']
Entire source index: [2, 5, 13, 11, 6, 175, 106, 9, 15, 75, 0, 4, 3]
Entire tokens: ['<sos>', 'ein', 'boston', 'terrier', 'läuft', 'über', 'saftig-grünes', 'gras', 'vor', 'einem', 'weißen', 'zaun', '.', '<eos>']
Entire source index: [2, 5, 3652, 4153, 83, 42, 0, 118, 27, 6, 51, 320, 4, 3]
Entire tokens: ['<sos>', 'ein', 'mädchen', 'in', 'einem', 'karateanzug', 'bricht', 'ein', 'brett', 'mit', 'einem', 'tritt', '.', '<eos>']
Entire source index: [2, 5, 25, 7, 6, 2897, 2784, 5, 1276, 11, 6, 402, 4, 3]
Entire tokens: ['<sos>', 'fünf', 'leute', 'in', 'winterjacken', 'und', 'mit', 'helmen', 'stehen', 'im', 'schnee', 'mit', 'schneemobilen', 'im', 'hintergrund', '.', '<eos>']
Entire source index: [2, 216, 73, 7, 5320, 10, 11, 827, 52, 20, 123, 11, 0, 20, 86, 4, 3]
Entire tokens: ['<sos>', 'leute', 'reparieren', 'das', 'dach', 'eines', 'hauses', '.', '<eos>']
Entire s

## C

In [ ]:
class CEncoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embed_dim)


        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio,bidirectional=True)
        

        self.dropout = nn.Dropout(dropout_ratio)

    
    def forward(self, src):

        embedded = self.dropout(self.embedding(src))


        outputs, (hidden, cell) = self.rnn(embedded)

        return outputs,hidden, cell


In [ ]:

class CDecoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()


        self.embedding = nn.Embedding(output_dim, embed_dim)


        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio,bidirectional=True)

        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim*2, output_dim)
        

        self.dropout = nn.Dropout(dropout_ratio)

        self.hidden_size = hidden_dim
        self.attn = nn.Linear(self.hidden_size * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)
        self.attn_reshapenn= nn.Linear(embed_dim+hidden_dim*2,embed_dim)
        self.attn_reshapenn2= nn.Linear(hidden_dim+hidden_dim,hidden_dim)

    def attention(self,encoder_outputs,hidden):
      timestep = encoder_outputs.size(0)
      h = hidden[0].repeat(timestep, 1, 1).transpose(0, 1)
      encoder_outputs = encoder_outputs.transpose(0, 1)
      encoder_outputs = self.attn_reshapenn2(encoder_outputs)
      
      energy = F.relu(self.attn(torch.cat([h, encoder_outputs], 2)))
      energy = energy.transpose(1, 2)  # [B*H*T]
      v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [B*1*H]
      energy = torch.bmm(v, energy).squeeze(1) 
      return F.softmax(energy, dim=1).unsqueeze(1)



    def forward(self, input,enc_output, hidden, cell):
        input = input.unsqueeze(0)

        
        embedded = self.dropout(self.embedding(input))


        dec_attn = self.attention(enc_output,hidden)
        
        context =dec_attn.bmm(enc_output.transpose(0, 1))
        context = torch.permute(context,(1,0,2))
        
        combined_input = torch.cat([context,embedded],dim=2)
        
        combined_input = self.attn_reshapenn(combined_input)
        
        output, (hidden, cell) = self.rnn(combined_input, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))

        return prediction, hidden, cell,dec_attn


In [ ]:

class CSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        enc_output,hidden, cell = self.encoder(src)


        trg_len = trg.shape[0] 
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden, cell,attention = self.decoder(input,enc_output, hidden, cell)
            outputs[t] = output 
            top1 = output.argmax(1) 
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 
        
        return outputs,attention


In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [ ]:
enc = CEncoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = CDecoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

cmodel = CSeq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
cmodel.apply(init_weights)

In [ ]:
optimizer = optim.Adam(cmodel.parameters())
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train() 
    epoch_loss = 0
    

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        trg =torch.permute(trg,(1,0))
        src =torch.permute(src,(1,0))
        output,attention = model(src, trg)
        

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)

        trg = trg[1:].reshape(-1)

        loss = criterion(output, trg)
        loss.backward() 
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval() 
    epoch_loss = 0
    
    with torch.no_grad():

        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            trg =torch.permute(trg,(1,0))
            src =torch.permute(src,(1,0))
            output,attention = model(src, trg, 0)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].reshape(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(cmodel, train_iterator, optimizer, criterion)
    valid_loss = evaluate(cmodel, valid_iterator, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(cmodel.state_dict(), 'Cseq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f}')

In [ ]:
example_idx = 6

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')
translation,attention = translate_sentence(src, SRC, TRG, cmodel, device)

print(f'predicted trg = {translation}')


src = ['ein', 'brauner', 'hund', 'rennt', 'dem', 'schwarzen', 'hund', 'hinterher', '.']
trg = ['a', 'brown', 'dog', 'is', 'running', 'after', 'the', 'black', 'dog', '.']


NameError: ignored

NameError: ignored